In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
legacy = pd.read_csv('/content/drive/My Drive/male_players (legacy).csv')
legacy.head()

<ipython-input-3-1ff297febb68>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy = pd.read_csv('/content/drive/My Drive/male_players (legacy).csv')


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


### Cleaning the Data

In [ ]:
legacy = legacy.drop(columns=['player_id', 'long_name', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date'])

In [ ]:
legacy = legacy.drop(columns= legacy.loc[:, "goalkeeping_speed":"player_face_url"].columns)

In [ ]:
legacy

,short_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,...,76,NaN,25,21,20,6,11,15,14,8
1,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,1985-02-05,185,80,...,85,NaN,22,31,23,7,11,15,14,11
2,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,1984-01-23,180,80,...,80,NaN,29,26,26,10,8,11,5,15
3,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,1981-10-03,195,95,...,91,NaN,25,41,27,13,15,10,9,12
4,M. Neuer,GK,90,90,63500000.0,300000.0,28,1986-03-27,193,92,...,37,NaN,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,Zhao Ziye,RW,46,61,110000.0,700.0,18,2003-10-01,180,73,...,54,40.0,23,21,25,9,13,13,12,7
161579,Wu Junjie,LB,46,58,110000.0,750.0,19,2003-06-02,188,83,...,31,35.0,50,51,45,6,14,8,13,14
161580,K. Barlow,CDM,46,58,110000.0,500.0,19,2003-06-28,181,73,...,37,35.0,36,45,50,8,9,7,14,9
161581,G. Tetteh,"RW, LW",46,70,150000.0,500.0,17,2005-05-20,175,68,...,63,43.0,19,17,14,13,12,14,7,13


In [ ]:
#dropping columns that have na values that are more than 30%
threshold = 0.3
number_of_nas = legacy.isna().sum()
percentage_of_nas = number_of_nas/len(legacy)
columns_to_drop = percentage_of_nas[percentage_of_nas > threshold].index
legacy.drop(columns=columns_to_drop, inplace=True)

In [ ]:
legacy

,short_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,...,76,NaN,25,21,20,6,11,15,14,8
1,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,1985-02-05,185,80,...,85,NaN,22,31,23,7,11,15,14,11
2,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,1984-01-23,180,80,...,80,NaN,29,26,26,10,8,11,5,15
3,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,1981-10-03,195,95,...,91,NaN,25,41,27,13,15,10,9,12
4,M. Neuer,GK,90,90,63500000.0,300000.0,28,1986-03-27,193,92,...,37,NaN,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,Zhao Ziye,RW,46,61,110000.0,700.0,18,2003-10-01,180,73,...,54,40.0,23,21,25,9,13,13,12,7
161579,Wu Junjie,LB,46,58,110000.0,750.0,19,2003-06-02,188,83,...,31,35.0,50,51,45,6,14,8,13,14
161580,K. Barlow,CDM,46,58,110000.0,500.0,19,2003-06-28,181,73,...,37,35.0,36,45,50,8,9,7,14,9
161581,G. Tetteh,"RW, LW",46,70,150000.0,500.0,17,2005-05-20,175,68,...,63,43.0,19,17,14,13,12,14,7,13


Grouping categorical and numerical columns

In [ ]:
from pandas.core.arrays import categorical
numeric_cols = legacy.select_dtypes(include=['int64','float64'])
categorical_cols = legacy.select_dtypes(include=['object'])

In [ ]:
numeric_cols

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_team_id,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93,95,100500000.0,550000.0,27,169,67,53.0,1.0,241.0,...,76,NaN,25,21,20,6,11,15,14,8
1,92,92,79000000.0,375000.0,29,185,80,53.0,1.0,243.0,...,85,NaN,22,31,23,7,11,15,14,11
2,90,90,54500000.0,275000.0,30,180,80,19.0,1.0,21.0,...,80,NaN,29,26,26,10,8,11,5,15
3,90,90,52500000.0,275000.0,32,195,95,16.0,1.0,73.0,...,91,NaN,25,41,27,13,15,10,9,12
4,90,90,63500000.0,300000.0,28,193,92,19.0,1.0,21.0,...,37,NaN,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46,61,110000.0,700.0,18,180,73,2012.0,1.0,112978.0,...,54,40.0,23,21,25,9,13,13,12,7
161579,46,58,110000.0,750.0,19,188,83,2012.0,1.0,112429.0,...,31,35.0,50,51,45,6,14,8,13,14
161580,46,58,110000.0,500.0,19,181,73,65.0,1.0,563.0,...,37,35.0,36,45,50,8,9,7,14,9
161581,46,70,150000.0,500.0,17,175,68,65.0,1.0,306.0,...,63,43.0,19,17,14,13,12,14,7,13


In [ ]:
categorical_cols

,short_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,nationality_name,preferred_foot,work_rate,body_type,real_face
0,L. Messi,CF,1987-06-24,La Liga,FC Barcelona,CF,2004-07-01,Argentina,Left,Medium/Low,Normal (170-),Yes
1,Cristiano Ronaldo,"LW, LM",1985-02-05,La Liga,Real Madrid CF,LW,2009-07-01,Portugal,Right,High/Low,Normal (185+),Yes
2,A. Robben,"RM, LM, RW",1984-01-23,Bundesliga,FC Bayern München,SUB,2009-08-28,Netherlands,Left,High/Low,Normal (170-185),Yes
3,Z. Ibrahimović,ST,1981-10-03,Ligue 1,Paris Saint-Germain,ST,2012-07-01,Sweden,Right,Medium/Low,Normal (185+),Yes
4,M. Neuer,GK,1986-03-27,Bundesliga,FC Bayern München,GK,2011-07-01,Germany,Right,Medium/Medium,Normal (185+),Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
161578,Zhao Ziye,RW,2003-10-01,Super League,Hebei CFFC,RES,2022-05-01,China PR,Right,Medium/Medium,Lean (170-185),No
161579,Wu Junjie,LB,2003-06-02,Super League,Guangzhou R&F,RES,2022-05-01,China PR,Left,Medium/Medium,Normal (185+),No
161580,K. Barlow,CDM,2003-06-28,Premier Division,Sligo Rovers,RES,2021-11-02,Republic of Ireland,Right,High/Medium,Normal (170-185),No
161581,G. Tetteh,"RW, LW",2005-05-20,Premier Division,Shamrock Rovers,RES,2022-06-24,Republic of Ireland,Right,Medium/Medium,Normal (170-185),No


Imputing with the mean

In [ ]:
imp = SimpleImputer(strategy = 'mean')

In [ ]:
df_filled = pd.DataFrame(imp.fit_transform(numeric_cols), columns=numeric_cols.columns)
df_filled

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_team_id,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,53.0,1.0,241.0,...,76.0,57.816892,25.0,21.0,20.0,6.0,11.0,15.0,14.0,8.0
1,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,53.0,1.0,243.0,...,85.0,57.816892,22.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,19.0,1.0,21.0,...,80.0,57.816892,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0
3,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,16.0,1.0,73.0,...,91.0,57.816892,25.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
4,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,19.0,1.0,21.0,...,37.0,57.816892,25.0,25.0,25.0,87.0,85.0,92.0,90.0,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,180.0,73.0,2012.0,1.0,112978.0,...,54.0,40.000000,23.0,21.0,25.0,9.0,13.0,13.0,12.0,7.0
161579,46.0,58.0,110000.0,750.0,19.0,188.0,83.0,2012.0,1.0,112429.0,...,31.0,35.000000,50.0,51.0,45.0,6.0,14.0,8.0,13.0,14.0
161580,46.0,58.0,110000.0,500.0,19.0,181.0,73.0,65.0,1.0,563.0,...,37.0,35.000000,36.0,45.0,50.0,8.0,9.0,7.0,14.0,9.0
161581,46.0,70.0,150000.0,500.0,17.0,175.0,68.0,65.0,1.0,306.0,...,63.0,43.000000,19.0,17.0,14.0,13.0,12.0,14.0,7.0,13.0


In [ ]:
categorical_cols.fillna(method='ffill')

,short_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,nationality_name,preferred_foot,work_rate,body_type,real_face
0,L. Messi,CF,1987-06-24,La Liga,FC Barcelona,CF,2004-07-01,Argentina,Left,Medium/Low,Normal (170-),Yes
1,Cristiano Ronaldo,"LW, LM",1985-02-05,La Liga,Real Madrid CF,LW,2009-07-01,Portugal,Right,High/Low,Normal (185+),Yes
2,A. Robben,"RM, LM, RW",1984-01-23,Bundesliga,FC Bayern München,SUB,2009-08-28,Netherlands,Left,High/Low,Normal (170-185),Yes
3,Z. Ibrahimović,ST,1981-10-03,Ligue 1,Paris Saint-Germain,ST,2012-07-01,Sweden,Right,Medium/Low,Normal (185+),Yes
4,M. Neuer,GK,1986-03-27,Bundesliga,FC Bayern München,GK,2011-07-01,Germany,Right,Medium/Medium,Normal (185+),Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
161578,Zhao Ziye,RW,2003-10-01,Super League,Hebei CFFC,RES,2022-05-01,China PR,Right,Medium/Medium,Lean (170-185),No
161579,Wu Junjie,LB,2003-06-02,Super League,Guangzhou R&F,RES,2022-05-01,China PR,Left,Medium/Medium,Normal (185+),No
161580,K. Barlow,CDM,2003-06-28,Premier Division,Sligo Rovers,RES,2021-11-02,Republic of Ireland,Right,High/Medium,Normal (170-185),No
161581,G. Tetteh,"RW, LW",2005-05-20,Premier Division,Shamrock Rovers,RES,2022-06-24,Republic of Ireland,Right,Medium/Medium,Normal (170-185),No


Encoding categorical variables

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    categorical_cols[col] = le.fit_transform(categorical_cols[col])
    label_encoders[col] = le

In [ ]:
categorical_cols

,short_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,nationality_name,preferred_foot,work_rate,body_type,real_face
0,25712,412,2854,15,478,3,48,6,0,7,3,1
1,8371,1128,2003,15,1013,14,249,134,1,1,5,1
2,3092,1541,1639,6,481,28,268,117,0,1,4,1
3,45328,1838,921,26,947,27,659,161,1,7,5,1
4,29784,719,2409,6,481,5,481,64,1,8,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
161578,45525,1643,8765,40,663,21,3942,34,1,8,1,0
161579,44350,721,8651,40,633,21,3942,34,0,8,5,0
161580,22834,289,8675,31,1165,21,3777,137,1,2,4,0
161581,15941,1694,9156,31,1139,21,3994,137,1,8,4,0


In [ ]:
new_data = pd.concat([df_filled, categorical_cols], axis=1)
new_data

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_team_id,...,dob,league_name,club_name,club_position,club_joined_date,nationality_name,preferred_foot,work_rate,body_type,real_face
0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,53.0,1.0,241.0,...,2854,15,478,3,48,6,0,7,3,1
1,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,53.0,1.0,243.0,...,2003,15,1013,14,249,134,1,1,5,1
2,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,19.0,1.0,21.0,...,1639,6,481,28,268,117,0,1,4,1
3,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,16.0,1.0,73.0,...,921,26,947,27,659,161,1,7,5,1
4,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,19.0,1.0,21.0,...,2409,6,481,5,481,64,1,8,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,180.0,73.0,2012.0,1.0,112978.0,...,8765,40,663,21,3942,34,1,8,1,0
161579,46.0,58.0,110000.0,750.0,19.0,188.0,83.0,2012.0,1.0,112429.0,...,8651,40,633,21,3942,34,0,8,5,0
161580,46.0,58.0,110000.0,500.0,19.0,181.0,73.0,65.0,1.0,563.0,...,8675,31,1165,21,3777,137,1,2,4,0
161581,46.0,70.0,150000.0,500.0,17.0,175.0,68.0,65.0,1.0,306.0,...,9156,31,1139,21,3994,137,1,8,4,0


Separating the independent and dependent variable

In [ ]:
y = new_data['overall']
X = new_data.drop('overall', axis=1)

Feature Extraction

In [ ]:
X_df =  pd.DataFrame(X)

In [ ]:
correlations = X_df.corrwith(y)
correlations   #finding the correlations of the 'overall' dataframe with every other variable

potential           0.695362
value_eur           0.563962
wage_eur            0.608711
age                 0.458390
height_cm           0.044415
                      ...   
nationality_name   -0.035847
preferred_foot     -0.042408
work_rate          -0.223711
body_type           0.123769
real_face           0.403125
Length: 67, dtype: float64

In [ ]:
#selecting features with correlations that are higher than 0.4 and less than negative 0.5
selected_features = correlations[(correlations > 0.4) | (correlations < -0.4)]
selected_features


potential                   0.695362
value_eur                   0.563962
wage_eur                    0.608711
age                         0.458390
international_reputation    0.485234
shooting                    0.448143
passing                     0.632617
dribbling                   0.563384
physic                      0.476732
attacking_short_passing     0.499757
skill_curve                 0.415863
skill_long_passing          0.483808
skill_ball_control          0.457011
movement_reactions          0.845753
power_shot_power            0.480297
power_long_shots            0.409086
mentality_vision            0.493282
mentality_composure         0.604074
real_face                   0.403125
dtype: float64

In [ ]:
#creating a dataframe for the selected features
X_selected = X[selected_features.index]
X_selected


,potential,value_eur,wage_eur,age,international_reputation,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure,real_face
0,95.0,100500000.0,550000.0,27.0,5.0,89.000000,86.000000,96.000000,63.000000,89.0,89.0,76.0,96.0,94.0,80.0,88.0,90.0,57.816892,1
1,92.0,79000000.0,375000.0,29.0,5.0,93.000000,81.000000,91.000000,79.000000,82.0,88.0,72.0,92.0,90.0,94.0,93.0,81.0,57.816892,1
2,90.0,54500000.0,275000.0,30.0,5.0,86.000000,83.000000,92.000000,64.000000,86.0,85.0,76.0,90.0,89.0,86.0,90.0,84.0,57.816892,1
3,90.0,52500000.0,275000.0,32.0,5.0,91.000000,81.000000,86.000000,86.000000,84.0,80.0,76.0,90.0,85.0,93.0,88.0,83.0,57.816892,1
4,90.0,63500000.0,300000.0,28.0,5.0,52.230764,56.938175,62.081872,64.814364,42.0,25.0,41.0,31.0,89.0,42.0,25.0,20.0,57.816892,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,61.0,110000.0,700.0,18.0,1.0,48.000000,40.000000,47.000000,44.000000,38.0,39.0,33.0,45.0,39.0,64.0,31.0,42.0,40.000000,0
161579,58.0,110000.0,750.0,19.0,1.0,25.000000,29.000000,34.000000,57.000000,30.0,24.0,25.0,32.0,42.0,33.0,25.0,31.0,35.000000,0
161580,58.0,110000.0,500.0,19.0,1.0,36.000000,43.000000,46.000000,53.000000,51.0,30.0,43.0,35.0,50.0,51.0,32.0,40.0,35.000000,0
161581,70.0,150000.0,500.0,17.0,1.0,50.000000,36.000000,46.000000,42.000000,40.0,38.0,26.0,43.0,45.0,42.0,47.0,44.0,43.000000,0


Scaling the independent variable

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

Training with Cross-Validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(129266, 19)

Cross-Validation for Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pickle as pkl

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred = rf_regressor.predict(X_test)
y_pred

array([61.3 , 63.72, 68.35, ..., 56.57, 63.53, 53.17])

Cross-Validation for XGBoost Regressor

In [ ]:
import xgboost as xgb

In [ ]:
xgb_regressor = xgb.XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
pred = xgb_regressor.predict(X_test)

Cross-Validation for Gradient Boost regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gb_plus_model = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.1, n_estimators=300, max_depth = 1, max_features = 5)
gb_plus_model.fit(X_train, y_train)

GradientBoostingRegressor(loss='absolute_error', max_depth=1, max_features=5,
                          n_estimators=300)

In [ ]:
pred_y = gb_plus_model.predict(X_test)

Evaluating the models

In [ ]:
from sklearn.metrics import mean_absolute_error

Calculating the MAE of the RandomForestRegressor

In [ ]:
mae_RF = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae_RF}")

Mean Absolute Error (MAE): 0.5046257387752576


Calculating the MAE of the XGBoostRegressor

In [ ]:
mae_XGB = mean_absolute_error(y_test, pred)
print(f"Mean Absolute Error (MAE): {mae_XGB}")

Mean Absolute Error (MAE): 0.6533553618681247


 Calculating the MAE of the GradientBoostingRegressor

In [ ]:
mae_GB = mean_absolute_error(y_test, pred_y)
print(f"Mean Absolute Error (MAE): {mae_GB}")

Mean Absolute Error (MAE): 1.2377471343815512


Fine Tuning the models then retesting

Fine Tuning for random forest

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
# Initializing the Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid,
                           cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print("Best Parameters:", best_params)

Fitting 3 folds for each of 810 candidates, totalling 2430 fits


KeyboardInterrupt: 

In [ ]:
y_pred = best_model.predict(X_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Fine tuning for xgboost

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror')

grid_search2 = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search2.fit(X_train, y_train)

Fitting 3 folds for each of 8748 candidates, totalling 26244 fits


KeyboardInterrupt: 

In [ ]:
best_params2 = grid_search2.best_params_
best_model2 = grid_search2.best_estimator_
print("Best parameters found: ", best_params)

In [ ]:
pred = best_model2.predict(X_test)
pred

Fine Tuning for Gradient Boost Regressor

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

gb_plus_model = GradientBoostingRegressor()

grid_search3 = GridSearchCV(estimator=gb_plus_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search3.fit(X_train, y_train)

In [ ]:
best_params3 = grid_search3.best_params_
best_model3 = grid_search3.best_estimator_
print("Best parameters found: ", best_params)

In [ ]:
pred_y = best_model3.predict(X_test)
pred_y

Preparing the dataset for testing

In [ ]:
players22 = pd.read_csv('/content/drive/My Drive/players_22-1.csv')
players22.head()

<ipython-input-44-a8bb603d7030>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players22 = pd.read_csv('/content/drive/My Drive/players_22-1.csv')


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
players22 = players22.drop(columns= players22.loc[:, "goalkeeping_speed":"nation_flag_url"].columns)

In [ ]:
#dropping all columns with na values more than 30%
threshold_2 = 0.3
number_of_nas2 = players22.isna().sum()
percentage_of_nas2 = number_of_nas2/len(players22)
columns_to_drop2 = percentage_of_nas2[percentage_of_nas2 > threshold_2].index
players22.drop(columns=columns_to_drop2, inplace=True)

In [ ]:
from pandas.core.arrays import categorical
numeric_testcols = players22.select_dtypes(include=['int64','float64'])
categorical_testcols = players22.select_dtypes(include=['object'])

In [ ]:
numeric_testcols

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,...,75,96,20,35,24,6,11,15,14,8
1,188545,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,...,90,88,35,42,19,15,6,12,8,10
2,20801,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,...,88,95,24,32,24,7,11,15,14,11
3,190871,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,...,93,93,35,32,29,9,9,15,15,11
4,192985,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,...,83,89,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,...,42,37,38,43,48,6,10,5,15,13
19235,262040,47,59,110000.0,500.0,19,175,70,445.0,1.0,...,44,47,37,44,47,11,12,6,8,10
19236,262760,47,55,100000.0,500.0,21,178,72,111131.0,1.0,...,39,36,38,44,48,8,6,7,10,6
19237,262820,47,60,110000.0,500.0,19,173,66,111131.0,1.0,...,49,47,10,14,11,7,10,7,14,15


In [ ]:
categorical_testcols = categorical_testcols.drop(columns=['long_name'])
categorical_testcols

,player_url,short_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,"RW, ST, CF",1987-06-24,Paris Saint-Germain,French Ligue 1,RW,2021-08-10,Argentina,Left,Medium/Low,Unique,Yes
1,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Poland,Right,High/Medium,Unique,Yes
2,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,"ST, LW",1985-02-05,Manchester United,English Premier League,ST,2021-08-27,Portugal,Right,High/Low,Unique,Yes
3,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Brazil,Right,High/Medium,Unique,Yes
4,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,"CM, CAM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Belgium,Right,High/High,Unique,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,https://sofifa.com/player/261962/defu-song/220002,Song Defu,CDM,1999-01-02,Wuhan FC,Chinese Super League,SUB,2021-04-01,China PR,Right,Medium/Medium,Lean (170-185),No
19235,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,CM,2002-03-18,Derry City,Rep. Ireland Airtricity League,RES,2021-02-22,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19236,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,CM,2000-02-07,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19237,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,ST,2002-04-23,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No


In [ ]:
#imputing the numeric using KNN imputation
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
numeric_testcols = pd.DataFrame(imputer.fit_transform(numeric_testcols), columns=numeric_testcols.columns)
numeric_testcols

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023.0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,1.0,...,75.0,96.0,20.0,35.0,24.0,6.0,11.0,15.0,14.0,8.0
1,188545.0,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,1.0,...,90.0,88.0,35.0,42.0,19.0,15.0,6.0,12.0,8.0,10.0
2,20801.0,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,1.0,...,88.0,95.0,24.0,32.0,24.0,7.0,11.0,15.0,14.0,11.0
3,190871.0,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,1.0,...,93.0,93.0,35.0,32.0,29.0,9.0,9.0,15.0,15.0,11.0
4,192985.0,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,1.0,...,83.0,89.0,68.0,65.0,53.0,15.0,13.0,5.0,10.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962.0,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,112541.0,1.0,...,42.0,37.0,38.0,43.0,48.0,6.0,10.0,5.0,15.0,13.0
19235,262040.0,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,445.0,1.0,...,44.0,47.0,37.0,44.0,47.0,11.0,12.0,6.0,8.0,10.0
19236,262760.0,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,111131.0,1.0,...,39.0,36.0,38.0,44.0,48.0,8.0,6.0,7.0,10.0,6.0
19237,262820.0,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,111131.0,1.0,...,49.0,47.0,10.0,14.0,11.0,7.0,10.0,7.0,14.0,15.0


In [ ]:
#imputing the categorical using forward fill
categorical_testcols.fillna(method='ffill')
categorical_testcols

,player_url,short_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,"RW, ST, CF",1987-06-24,Paris Saint-Germain,French Ligue 1,RW,2021-08-10,Argentina,Left,Medium/Low,Unique,Yes
1,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Poland,Right,High/Medium,Unique,Yes
2,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,"ST, LW",1985-02-05,Manchester United,English Premier League,ST,2021-08-27,Portugal,Right,High/Low,Unique,Yes
3,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Brazil,Right,High/Medium,Unique,Yes
4,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,"CM, CAM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Belgium,Right,High/High,Unique,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,https://sofifa.com/player/261962/defu-song/220002,Song Defu,CDM,1999-01-02,Wuhan FC,Chinese Super League,SUB,2021-04-01,China PR,Right,Medium/Medium,Lean (170-185),No
19235,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,CM,2002-03-18,Derry City,Rep. Ireland Airtricity League,RES,2021-02-22,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19236,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,CM,2000-02-07,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19237,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,ST,2002-04-23,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#encoding the categorical columns
label_encoders2 = {}
for col in categorical_testcols:
    le2 = LabelEncoder()
    categorical_testcols[col] = le2.fit_transform(categorical_testcols[col])
    label_encoders2[col] = le2

In [ ]:
categorical_testcols

,player_url,short_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,196,10117,604,719,470,19,25,1808,6,0,7,9,1
1,1235,14468,635,1016,241,21,27,210,123,1,2,9,1
2,3613,3288,658,266,412,17,27,1825,124,1,1,9,1
3,1520,13097,372,2143,470,19,14,652,20,1,2,9,1
4,1730,9005,168,1940,411,17,19,353,13,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,17670,16139,98,4570,688,6,28,1682,30,1,8,1,0
19235,17705,2947,167,5699,212,39,21,1645,126,1,8,1,0
19236,18088,12809,167,4958,291,39,21,1755,126,1,8,1,0
19237,18123,10276,635,5735,291,39,21,1755,126,1,8,1,0


In [ ]:
new_df = pd.concat([numeric_testcols, categorical_testcols], axis=1)
new_df

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,158023.0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,1.0,...,719,470,19,25,1808,6,0,7,9,1
1,188545.0,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,1.0,...,1016,241,21,27,210,123,1,2,9,1
2,20801.0,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,1.0,...,266,412,17,27,1825,124,1,1,9,1
3,190871.0,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,1.0,...,2143,470,19,14,652,20,1,2,9,1
4,192985.0,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,1.0,...,1940,411,17,19,353,13,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962.0,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,112541.0,1.0,...,4570,688,6,28,1682,30,1,8,1,0
19235,262040.0,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,445.0,1.0,...,5699,212,39,21,1645,126,1,8,1,0
19236,262760.0,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,111131.0,1.0,...,4958,291,39,21,1755,126,1,8,1,0
19237,262820.0,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,111131.0,1.0,...,5735,291,39,21,1755,126,1,8,1,0


In [ ]:
y2 = new_df['overall']
X2 = new_df.drop('overall', axis=1)

In [ ]:
#extracting the corresponding features from the legacy dataset
testdata = new_df[selected_features.index]

In [ ]:
testdata

,potential,value_eur,wage_eur,age,international_reputation,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure,real_face
0,93.0,78000000.0,320000.0,34.0,5.0,92.0,91.0,95.0,65.0,91.0,93.0,91.0,96.0,94.0,86.0,94.0,95.0,96.0,1
1,92.0,119500000.0,270000.0,32.0,5.0,92.0,79.0,86.0,82.0,85.0,79.0,70.0,88.0,93.0,90.0,87.0,81.0,88.0,1
2,91.0,45000000.0,270000.0,36.0,5.0,94.0,80.0,88.0,75.0,80.0,81.0,77.0,88.0,94.0,94.0,93.0,76.0,95.0,1
3,91.0,129000000.0,270000.0,29.0,5.0,83.0,86.0,94.0,63.0,86.0,88.0,81.0,95.0,89.0,80.0,81.0,90.0,93.0,1
4,91.0,125500000.0,350000.0,30.0,4.0,86.0,93.0,88.0,78.0,94.0,85.0,93.0,91.0,91.0,91.0,91.0,94.0,89.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,52.0,70000.0,1000.0,22.0,1.0,35.0,46.0,48.0,49.0,50.0,33.0,48.0,49.0,53.0,46.0,30.0,43.0,37.0,0
19235,59.0,110000.0,500.0,19.0,1.0,39.0,50.0,46.0,51.0,51.0,53.0,50.0,42.0,49.0,49.0,38.0,49.0,47.0,0
19236,55.0,100000.0,500.0,21.0,1.0,37.0,45.0,49.0,52.0,49.0,37.0,49.0,49.0,46.0,50.0,34.0,46.0,36.0,0
19237,60.0,110000.0,500.0,19.0,1.0,46.0,36.0,48.0,42.0,38.0,36.0,33.0,45.0,48.0,48.0,41.0,40.0,47.0,0


In [ ]:
#scaling the independent variables
X2_scaled = scaler.fit_transform(testdata)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,
                                       test_size=0.25,
                                       random_state=42)
X2_train.shape, X2_test.shape

((14429, 69), (4810, 69))

Testing using the random forest regressor

In [ ]:
finalpredfor_y = rf_regressor.predict(X2_scaled)
finalpredfor_y

array([92.63, 91.76, 90.81, ..., 52.89, 52.57, 52.82])

In [ ]:
#calculating the mean absolute error
mae_RF2 = mean_absolute_error(y2, finalpredfor_y)
print(f"Mean Absolute Error (MAE): {mae_RF2}")

Mean Absolute Error (MAE): 0.8177207755080826


In [ ]:
RF_bestmodel = rf_regressor

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
import pickle as pkl
# saving the model in a pickle file
with open('rf_model.pkl', 'wb') as file:
    pkl.dump(RF_bestmodel, file)